Rong Wang, rongwww, 1619779944

<b>1. Time Series Classification Part 1: Feature Creation/Extraction</b>

An interesting task in machine learning is classification of time series. In this problem, we will classify the activities of humans based on time series obtained by a Wireless Sensor Network.

(a) Download the AReM data from: https://archive.ics.uci.edu/ml/datasets/ Activity+Recognition+system+based+on+Multisensor+data+fusion+\%28AReM\ %29 . The dataset contains 7 folders that represent seven types of activities. In each folder, there are multiple files each of which represents an instant of a human performing an activity.1 Each file containis 6 time series collected from activities
of the same person, which are called avg rss12, var rss12, avg rss13, var rss13, vg rss23, and ar rss23. There are 88 instances in the dataset, each of which con- tains 6 time series and each time series has 480 consecutive values.

(b) Keep datasets 1 and 2 in folders bending1 and bending 2, as well as datasets 1, 2, and 3 in other folders as test data and other datasets as train data.

In [153]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import collections
from astropy.stats import bootstrap
import statistics
from numpy import mean
from numpy import median
from numpy import percentile
import scipy as sp
import scipy.stats

In [154]:
dir_path = '../data/AReM/'
folders = ['bending1', 'bending2', 'cycling', 'lying', 'sitting', 'standing', 'walking']
col_name = ['time', 'avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']
# 69 train datasets
traindata_path_num = [[3, 4, 5, 6, 7], 
                      [3, 4, 5, 6], 
                      [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
                      [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
                      [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
                      [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
                      [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]]
# 19 test datasets
testdata_path_num = [[1, 2], [1, 2], [1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3]]

trainset, testset = [], []

# train data
for folder_num in range(len(folders)):
    folder_name = folders[folder_num]
    dataset_num = traindata_path_num[folder_num]
    for i in dataset_num:
        file_path = dir_path + folder_name + "/" + "dataset" + str(i) + ".csv"
        if file_path == '../data/AReM/bending2/dataset4.csv':
            dataframe = pd.read_csv(file_path, skiprows=5, header=None, delim_whitespace=True, names=col_name)
        else:
            dataframe = pd.read_csv(file_path, skiprows=5, encoding='utf-8', names=col_name)
        trainset.append(dataframe)
#         print(file_path + " size: " + str(dataframe.shape))
    
# test data
for folder_num in range(len(folders)):
    folder_name = folders[folder_num]
    dataset_num = testdata_path_num[folder_num]
    for i in dataset_num:
        file_path = dir_path + folder_name + "/" + "dataset" + str(i) + ".csv"
        dataframe = pd.read_csv(file_path, skiprows=5, encoding='utf-8', names=col_name)
        testset.append(dataframe)
#         print(file_path + " size: " + str(dataframe.shape))          
            
print("trainset size: " + str(len(trainset)) + " lines " + str(trainset[0].shape))
print("testset size: " + str(len(testset)) + " lines " + str(testset[0].shape))          

trainset size: 69 lines (480, 7)
testset size: 19 lines (480, 7)


(c) Feature Extraction
Classification of time series usually needs extracting features from them. In this problem, we focus on time-domain features.

i. Research what types of time-domain features are usually used in time series classification and list them (examples are minimum, maximum, mean, etc).

minimum, maximum, mean, median, standard deviation, first quartile, third quartile, range, skewness, kurtosis.

ii. Extract the time-domain features minimum, maximum, mean, median, stan- dard deviation, first quartile, and third quartile for all of the 6 time series in each instance. You are free to normalize/standardize features or use them directly.

For example, 1st quart6, means the first quartile of the sixth time series in each of the 88 instances.

In [155]:
# features = ['min', 'max', 'mean', 'median', 'std', '1st_quart', '3rd_quart']
stat = collections.defaultdict(dict)
ind = 0
for datafile in trainset+testset: # one datafile is one instance, is one dataset file.
    ind += 1
    for i, time_series_col in enumerate(col_name[1:]): # ['avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']
        series_data = datafile[time_series_col].describe()
        series_num = i+1
        stat[ind]['min' + str(series_num)] = series_data['min']
        stat[ind]['max' + str(series_num)] = series_data['max']
        stat[ind]['mean' + str(series_num)] = series_data['mean']
        stat[ind]['median' + str(series_num)] = series_data['50%']
        stat[ind]['std' + str(series_num)] = series_data['std']
        stat[ind]['1st_quartile' + str(series_num)] = series_data['25%']
        stat[ind]['3rd_quartile' + str(series_num)] = series_data['75%']
stat_table = pd.DataFrame(stat).T
stat_table # first 69 lines are for trainset, last 19 lines are for testset.

,min1,max1,mean1,median1,std1,1st_quartile1,3rd_quartile1,min2,max2,mean2,...,std5,1st_quartile5,3rd_quartile5,min6,max6,mean6,median6,std6,1st_quartile6,3rd_quartile6
1,35.00,47.40,43.954500,44.330,1.558835,43.00,45.0000,0.0,1.70,0.426250,...,1.999604,35.3625,36.500,0.0,1.79,0.493292,0.430,0.513506,0.000,0.940
2,33.00,47.75,42.179813,43.500,3.670666,39.15,45.0000,0.0,3.00,0.696042,...,3.849448,30.4575,36.330,0.0,2.18,0.613521,0.500,0.524317,0.000,1.000
3,33.00,45.75,41.678063,41.750,2.243490,41.33,42.7500,0.0,2.83,0.535979,...,2.411026,28.4575,31.250,0.0,1.79,0.383292,0.430,0.389164,0.000,0.500
4,37.00,48.00,43.454958,43.250,1.386098,42.50,45.0000,0.0,1.58,0.378083,...,2.488862,22.2500,24.000,0.0,5.26,0.679646,0.500,0.622534,0.430,0.870
5,36.25,48.00,43.969125,44.500,1.618364,43.31,44.6700,0.0,1.50,0.413125,...,3.318301,20.5000,23.750,0.0,2.96,0.555313,0.490,0.487826,0.000,0.830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,35.50,46.25,43.174938,43.670,1.989052,42.50,44.5000,0.0,2.12,0.506583,...,2.983976,12.7500,16.500,0.0,5.72,0.911979,0.830,0.666161,0.470,1.220
85,32.75,47.00,42.760563,44.500,3.398919,41.33,45.3725,0.0,3.34,0.486167,...,4.296574,13.0000,18.565,0.0,5.73,0.842271,0.710,0.722165,0.430,1.090
86,19.33,43.50,34.227771,35.500,4.889576,30.50,37.7500,0.0,14.50,3.995729,...,3.092094,14.7500,18.670,0.0,9.74,3.394125,3.100,1.792090,2.105,4.425
87,12.50,45.00,33.509729,34.125,4.850923,30.50,36.7500,0.0,13.05,4.450771,...,3.133564,14.6275,18.750,0.0,8.96,3.378479,3.085,1.787360,2.060,4.440


iii. Estimate the standard deviation of each of the time-domain features you extracted from the data. Then, use Python’s bootstrapped or any other method to build a 90% bootsrap confidence interval for the standard deviation of each feature.

In [156]:
# st_dev=np.std(stat_table) 
# print(st_dev)

std_stat = collections.defaultdict(dict)
for feature in stat_table.columns:
    std_stat[feature]["standard_deviation"] = np.std(stat_table[feature])
#     print('The standard deviation of ' + feature + ' is ' + str(np.std(stat_table[feature])))
    feature_array = np.array(stat_table[feature])
    n = len(feature_array)
    bootstrap = []
    for i in range(1000):
        ind = np.random.randint(0, n, size=n)
        data_sample = feature_array[ind]
        bootstrap.append(np.std(data_sample))
    bootstrap.sort()
    a = 1 - 0.9
    std_stat[feature]["90%_bootstrap_confidence_interval"] = '[' + str(bootstrap[int(1000 * a / 2)]) + ', ' + str(bootstrap[int(1000 * (1 - a / 2))]) + ']'
#     print('90% bootstrap confidence interval is: [' + str(bootstrap[int(1000 * a / 2)]) + ', ' + str(bootstrap[int(1000 * (1 - a / 2))]) + ']')
std_stat_table = pd.DataFrame(std_stat).T
std_stat_table

,standard_deviation,90%_bootstrap_confidence_interval
min1,9.515445,"[8.208716366166229, 10.72700749323007]"
max1,4.369322,"[3.2458553594488477, 5.307806963314891]"
mean1,5.305314,"[4.709658173537759, 5.861402707264918]"
median1,5.409056,"[4.743047416911437, 5.942663695258046]"
std1,1.762056,"[1.5542719415749657, 1.9266113285432824]"
1st_quartile1,6.118526,"[5.50681057823348, 6.637550324901437]"
3rd_quartile1,5.109643,"[4.217848403812224, 5.775157647339774]"
min2,0.0,"[0.0, 0.0]"
max2,5.033882,"[4.599466179892982, 5.381775678387139]"
mean2,1.565194,"[1.3981886989863266, 1.6944130542032474]"


iv. Use your judgement to select the three most important time-domain features (one option may be min, mean, and max).

1. Mean: to capture the average values of the time series
2. First Quartile: to see lower bound values of the series
3. Max: to utilise the peak values of the time series

<b>2. ISLR 3.7.4</b>

I collect a set of data (n = 100 observations) containing a single predictor and a quantitative response. I then fit a linear regression model to the data, as well as a separate cubic regression, i.e. Y = β0 +β1X +β2X2 +β3X3 +ε.

(a) Suppose that the true relationship between X and Y is linear, i.e. Y = β0 + β1X + ε. Consider the training residual sum of squares (RSS) for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

Cubic regression is better. Because cubic regression is more flexible than linear model, when it only has 100 obseavations, cubic regression will get a good fitting than linear regression. When we use polynomial features, we get more parameters, and the generated curve will try to fit more data points than a linear line.

(b) Answer (a) using test rather than training RSS.

I would expect the linear regression to perform slightly better than the cubic regression. We may have overfitting training on 100 observations, so cubic regression may be less accurate when it predicts the unseen data.

(c) Suppose that the true relationship between X and Y is not linear, but we don’t know how far it is from linear. Consider the training RSS for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

Cubic regression will has a lower training RSS. Since we don't know how far it is from line, so here more parameters mean the regression model will be more flwxible, is able to fit more data points.

(d) Answer (c) using test rather than training RSS.

I think it would depends on the real relationship between X and Y. If it is a strong non-linear relationship, then we would obviously expect the cubic regression to have a lower test RSS. But if it is close to linear, or maybe a mildly non-linear relationship, linear model may perform better. 